<a href="https://colab.research.google.com/github/adidror005/youtube-videos/blob/main/TradingStream2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

### Install Python SDK

### Grab API key and Secret

### Connect to Trading Client

### Helper Function to View Respones as DataFrames

In [ ]:
import pandas as pd

class Util:
    @staticmethod
    def to_dataframe(data):
        if isinstance(data, list):
            return pd.DataFrame([item.__dict__ for item in data])
        return pd.DataFrame(data, columns=['tag', 'value']).set_index('tag')

### Cancel all Orders

# Make a Buy Order for 1000$ of AMD

In [1]:
from alpaca.trading.requests import MarketOrderRequest,TakeProfitRequest,LimitOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce, OrderClass




## Let's look at open orders

In [ ]:
from alpaca.trading.enums import QueryOrderStatus
from alpaca.trading.requests import GetOrdersRequest
Util.to_dataframe(trading_client.get_orders(GetOrdersRequest(status=QueryOrderStatus.OPEN)))

,id,client_order_id,created_at,updated_at,submitted_at,filled_at,expired_at,expires_at,canceled_at,failed_at,...,limit_price,stop_price,status,extended_hours,legs,trail_percent,trail_price,hwm,position_intent,ratio_qty
0,4954b422-9158-4974-8e8c-0b04dd54b29e,3bbe2ed9-fa9d-4629-acdb-db757d2950a2,2025-05-12 09:35:06.355465+00:00,2025-05-12 09:35:06.365818+00:00,2025-05-12 09:35:06.364437+00:00,None,None,2025-05-13 00:00:00+00:00,None,None,...,114.3,None,OrderStatus.NEW,True,None,None,None,None,PositionIntent.BUY_TO_OPEN,None
